In [ ]:
import pandas as pd
import numpy as np
import os
import json

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Set up paths
raw_folder = './raw_data/'
tmp_folder = './tmp_data/'
processed_folder = './processed_data/'

imdb_rating_folder = raw_folder + 'imdb/'
cmu_folder = raw_folder + 'cmu/'
baby_names_folder = raw_folder + 'baby_names_national/'

# Data preprocessing
In this notebook we aim to select the relevant features for our model and to clean the data. This also includes merging content from the different datasets, from various sources such as the `IMDB` datasets, the baby names datasets, etc... which have been either downloaded or scraped from the web.

**Note on structure:** We separate dataframes into various folders, raw, tmp (temporary data) and processed. 
- The raw data is what we download from the web, including the initial CMU datasets. These are unmodified content that we use as a basis for our work, it is read-only.
- The temporary (tmp) data is the data that we use to work on, and that we modify. This is often the case for scrapped data from the web, or dataframes that are being used in a different notebook other than processing.
- We put our final cleaned data in the processed folder, which is the data that we use for our models.

## 1. Initial view on the data
### a. CMU datasets
Our project requires the use of the `character.metadata.tsv` and `movie.metadata.tsv` files from the CMU databases.


In [ ]:
# Import character metadata
character_df = pd.read_csv(cmu_folder + 'character.metadata.tsv', sep='\t', header=None)

# Add column names deduced from README
character_df.columns = ['wiki_ID', 'free_ID', 'release', 'char_name', 'DOB', 'gender', 'height', 'ethnicity', 'act_name', 'age_at_release', 'free_char_map1', 'free_char_map2', 'free_char_map3']

display(character_df.head())
print(f"There are {len(character_df)} characters in the dataset.")

In [ ]:
# Import movie metadata
movie_df = pd.read_csv(cmu_folder + 'movie.metadata.tsv', sep='\t', header=None)

# Add column names deduced from README
movie_df.columns = ['wiki_ID', 'free_ID', 'mov_name', 'release', 'revenue', 'runtime', 'languages', 'countries', 'genres']
display(movie_df.head(5))
print(f"There are {len(movie_df)} movies in the dataset.")
print(f"Is the wiki_ID unique? {movie_df.wiki_ID.is_unique}")

Great, we can use the wikipedia IDs as unique identifers for the movies!

We immediately see that there are some columns that we won't need in our analysis, which we will drop later on, such as the freebase character maps, releases in the characters dataframe (because they are already in the movies dataframe), etc...

We also see that there are some missing values for `revenues`, we need to take note of how any of them are missing to see if they would affect our analysis.

In [ ]:
nb_revenue_missing = movie_df['revenue'].isna().sum()
total_movies = len(movie_df)
perc_missing = (nb_revenue_missing / total_movies)*100
print(f"Percentage of missing values in column 'revenue': {perc_missing:.2f}%")

### b. IMDB datasets: Ratings
We would like to use ratings and rating counts to assess the popularity of a movie. We will use the `imdb_ratings.tsv` file from the [IMDB datasets](https://datasets.imdbws.com/).

In [ ]:
# Import imdb data
rating_df = pd.read_csv(imdb_rating_folder + 'imdb_rating.tsv', sep='\t', index_col='tconst')
display(rating_df.head())

In [ ]:
# Verify the indexes are unique
print(f"Is the indexing unique ? {rating_df.index.is_unique}")

`tconst` is a unique identifer in the IMDB database, which we can use to merge with our other dataframes. We need to figure out how to merge these dataframes together.

### c. Baby names data
We consider the names of babies across the years in the USA to be a good indicator of the popularity of a name. We will use the `NationalNames.csv` file from the [US Social Security Administration](https://www.ssa.gov/oact/babynames/limits.html), and will be put in the `baby_names_national` folder. This dataset has all the names of babies born in the USA from 1880 to 2023, where each count is the number of babies born with that name in that year.

We need to iterate over all the years to create a dataframe with the counts of all the names, for all years:
- `name`: name of the baby
- `year`: year of birth
- `number`: number of babies born with that name in that year 
- `percentage`: percentage of babies born with that name in that year, for normalization purposes (added later in the notebook)

In [ ]:
# Define the path to your dataset folder
folder_path = 'raw_data/baby_names_national/'

# Create an empty list to store individual DataFrames
data_frames = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.startswith('yob') and filename.endswith('.txt'):
        # Extract the year from the filename
        year = int(filename[3:-4])

        # Read the data from the file into a DataFrame
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, header=None, names=['name', 'gender', 'number'])

        # Add the 'year' column to the DataFrame
        df['year'] = year

        # Append the current DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames in the list into one DataFrame
baby_name_df = pd.concat(data_frames, ignore_index=True)

# Set the 'name' column as the index
# combined_data.set_index('name', inplace=True)

# Display the resulting DataFrame
baby_name_df.head()

## 2. Data processing and cleaning
In this section we modify some of the dataframes values to make them usable for our analysis, and populate them with extra data from other sources.

### a. Linking external movie IDs (Scraping part 1)
One of our main sources of data is the [IMDB](https://www.imdb.com/) and the [TMDB](https://www.themoviedb.org/) website. To be able to do so, we need to find a link from our CMU IDs (wikipedia) to the IMDB and TMDB movie IDs. We've explored 2 methods of collecting this kind of data. All the process is extensively documented in the `convert_ds.ipynb` notebook. 

Therefore make sure you run [this notebook](./scraping/convert_ids.ipynb) before continuing. It should have generated the `movies_external_ids.csv` file in the `tmp` folder, which we will use in the following steps.

In [ ]:
# Test import with a try
try:
    df = pd.read_csv(tmp_folder + 'movies_external_ids.csv', sep='\t', header=None)
except:
    raise FileNotFoundError("Please run the ID scrapping notebook first.")

### b. Movie metadata preprocessing
#### Release date column
The release date column is very important for our study. Let's compute the missing values and remove the NaN value in the release column, we will drop these rows later on.

# <h1 style="font-size: 50px; color: red">⚠️**TODO:** change structure instead of `release` to `year` and `month` columns. If no month, just leave nan!</h1>

In [ ]:
nb_release_missing = movie_df['release'].isna().sum()
total_movies = len(movie_df)
perc_missing = (nb_release_missing / total_movies)*100
print(f"Percentage of missing values in column 'release': {perc_missing:.2f}%")

. The movies release has a weird format. There are 3 different formats:
- of length 10: `YYYY-MM-DD`
- of length 7: `YYYY-MM`
- of length 4: `YYYY`

In [ ]:
counts_lengths = movie_df['release'].dropna().apply(lambda x: len(str(x))).value_counts()
print(f"In the release column of movie_df the string have the following lengths with their frequency : \n\n{counts_lengths}")

In [ ]:
length4_test = movie_df[movie_df['release'].apply(lambda x: len(str(x)) == 4)].iloc[0]['release']
print(f"Example of a value of length 4: {length4_test}")
length7_test = movie_df[movie_df['release'].apply(lambda x: len(str(x)) == 7)].iloc[0]['release']
print(f"Example of a value of length 7: {length7_test}")
length10_test = movie_df[movie_df['release'].apply(lambda x: len(str(x)) == 10)].iloc[0]['release']
print(f"Example of a value of length 10: {length10_test}")

We should convert the release date to keep only the year as we have only the year in the baby name dataset. To do so we simply take the first 4 characters of the release date, which is the year. We then convert it to an integer.

In [ ]:
# Drop rows with NaN values in the release column
movie_datetime_df = movie_df.dropna(subset=['release']).copy(deep=True)

# Take only year
movie_datetime_df['release'] = movie_datetime_df['release'].apply(lambda x: str(x)[:4]).astype(np.int64)

# Remove outliers (years before 1800, there is one that is at 1010)
movie_datetime_df = movie_datetime_df[movie_datetime_df['release'] >= 1800]

movie_datetime_df[['wiki_ID', 'mov_name', 'release']].head()

In [ ]:
# Reset the movie_df to the datetime version
movie_df = movie_datetime_df.copy(deep=True)

#### Add movie ratings and count

In [ ]:
# Import idmb with wikidata index mapping
movies_to_imdb_id_df = pd.read_csv(tmp_folder + 'movies_external_ids.csv')
display(movies_to_imdb_id_df)

Let's check if we have the rating of all the movies. To do so we carry out a left merge and count the number of missing values:

In [ ]:
left_merged_imdb_movie_df = pd.merge(movie_df.reset_index(), movies_to_imdb_id_df, left_on='wiki_ID', right_on='wikipedia_ID', how='left').copy(deep=True)
display(left_merged_imdb_movie_df.head(2))
print(f"Length of the dataframe : {len(left_merged_imdb_movie_df)}")

In [ ]:
nb_rating_missing = left_merged_imdb_movie_df['IMDB_ID'].isna().sum()
total_movies = len(left_merged_imdb_movie_df)
perc_missing = (nb_rating_missing / total_movies)*100
print(f"Percentage of missing values in column 'averageRating': {perc_missing:.2f}%")

We miss about 5% of the IMDB index. Therefore, the number of movie is reduced of 5% as well. Let's make the inner merge to keep only the movie for which we know the rating

In [ ]:
merged_imdb_movie_df = pd.merge(movie_df.reset_index(), movies_to_imdb_id_df, left_on='wiki_ID', right_on='wikipedia_ID', how='inner').copy(deep=True)
display(merged_imdb_movie_df.head(2))
print(f"Length of the dataframe : {len(merged_imdb_movie_df)}")

merged_rating_movie_df = pd.merge(merged_imdb_movie_df, rating_df, left_on='IMDB_ID', right_on='tconst', how='inner').copy(deep=True)
display(merged_rating_movie_df.head(2))
print(f"Length of the dataframe : {len(merged_rating_movie_df)}")

Lets remove the columns that we don't need

In [ ]:
movie_rating_df = merged_rating_movie_df[['wiki_ID', 'mov_name', 'release', 'revenue', 'genres', 'numVotes', 'averageRating']].copy(deep=True)
display(movie_rating_df.head(2))
print(f"length of the dataframe : {len(movie_rating_df)}")

In [ ]:
movie_df = movie_rating_df.copy(deep=True)

### Genre column cleaning

In [ ]:
print(f"Are there movies with missing genre fields? {movie_df['genres'].isna().sum() != 0}")

In the genre column we have dictionaries. The key is a freebase value, and the value is the associated string. So we'll keep the string only, and we'll end up with a simple list of genres.

We want to generate an aditional dataset that has the wikiID of a movie and the genre asosciated with that movie.

In [ ]:
def genres_convert(genres_raw):
    """Converts the genres column from a string to a list of genres
    """
    genres_list = []
    genres_raw = json.loads(genres_raw)

    # Keep only values
    for genre in genres_raw.values():
        genres_list.append(genre)

    return genres_list

In [ ]:
# Transform genres column from json string to list
movies_processed_genres = movie_df.copy(deep=True)
movies_processed_genres['genres'] = movies_processed_genres['genres'].apply(genres_convert)

# Create a new DataFrame with each genre in a separate row
expanded_genres = pd.DataFrame(columns=['wiki_ID', 'genre'])

# We don't directly add rows to the DataFrame because it is inefficient, so we create lists to store the values
exp_wiki_ids = []
exp_genres = []
for index, row in movies_processed_genres.iterrows():
    # Extract wikiID and genres dictionary
    wiki_id = row['wiki_ID']
    genre_list = row['genres']

    # Iterate through each genre in the genres dictionary
    for genre in genre_list:
        # Append each genre as a new row to the expanded_genres DataFrame
        exp_wiki_ids += [wiki_id]
        exp_genres += [genre]

expanded_genres['wiki_ID'] = exp_wiki_ids
expanded_genres['genre'] = exp_genres

# Set index to wikiID
expanded_genres.set_index('wiki_ID', inplace=True)
expanded_genres.sort_index(inplace=True)
expanded_genres.groupby('genre').count().sort_values(by='genre', ascending=False)

print(f"Number of different genres: {len(expanded_genres['genre'].unique())}")

display(expanded_genres.head())
print(f"Length of the genres dataframe : {len(expanded_genres)}")

In [ ]:
# Save the processed dataframe
expanded_genres.to_csv(processed_folder + 'movie_genres_df.csv')

# Remove the genres column from the movie_df
movie_df.drop(columns=['genres'], inplace=True)
display(movie_df.head(2))

Let's do a simple data exploration on the genres of the movies:

In [ ]:
# Plot the number of movies per genre
plt.figure(figsize=(10, 12))
sns.countplot(y='genre', data=expanded_genres, order=expanded_genres['genre'].value_counts().index[:40]) # only take top 40 genres

plt.title('Number of Movies per Genre')
plt.xlabel('Number of Movies')
plt.ylabel('Genre')
plt.show()

# Count the number of genres in movies
genre_counts = expanded_genres.groupby('wiki_ID')['genre'].count()
genre_count_frequency = genre_counts.value_counts().sort_index()

genre_count_frequency.plot(kind='bar')
plt.title('Number of Genres per Movie')
plt.xlabel('Number of Genres')
plt.ylabel('Number of Movies')
plt.yscale('log')
plt.show()

### c. Baby name data preprocessing
First we combine the rows with the same name and same year in order to ignore the gender in the dataset: we study the overall impact of the name regardless of the gender of the baby

In [ ]:
baby_name_filtered_df = baby_name_df.groupby(['name', 'year'])['number'].sum().to_frame()
display(baby_name_filtered_df)

For each datapoint, compute the percentage of the total births in the year. We do this to normalize the data, in case the number of births per year has increased over the years or if a year had notably more births than another.

In [ ]:
# group the baby name dataframe by year to get the number of birth per year.
birth_per_year_df = baby_name_filtered_df.groupby('year')['number'].sum().to_frame()
birth_per_year_df.reset_index(inplace=True)

birth_per_year_df = birth_per_year_df.rename(columns={'number': 'total_number'})

print("birth_per_year_df:")
display(birth_per_year_df.head(3))

# Merge dataframes
merged_df = pd.merge(baby_name_filtered_df.reset_index(), birth_per_year_df, on='year')

# Calculate the percentage and add it as a new column to dataframe
merged_df['percentage'] = (merged_df['number'] / merged_df['total_number']) * 100

print("merged_df:")
display(merged_df.head(3))

baby_name_with_percentage_df = merged_df.drop('total_number', axis=1)
baby_name_with_percentage_df.set_index(['name', 'year'], inplace=True)

print("baby_name_with_percentage_df:")
display(baby_name_with_percentage_df.head(3))

### d. Character metadata preprocessing
The character dataframe has not a unique index. It is due to the several NaN values present in the same movie. To tackles this we can drop the rows that have a NaN as name and see if it solve the issue.

In [ ]:
character_filtered = character_df.copy(deep=True)

# Drop the rows with a NaN as the character name
character_filtered = character_filtered.dropna(subset=['char_name'])

# Drop the rows with the same character name
character_filtered = character_filtered.drop_duplicates(subset=['wiki_ID','char_name','gender'])
character_filtered = character_filtered.set_index(['wiki_ID','char_name', 'gender'])

display(character_filtered.head())

We drop all columns that are not interesting for us:

In [ ]:
columns_to_drop = ['free_ID', 'release', 'ethnicity', 'free_char_map1', 'free_char_map2', 'free_char_map3']
columns_to_drop_existing = [col for col in columns_to_drop if col in character_filtered.columns]

character_filtered.drop(columns_to_drop_existing, axis=1, inplace=True)

display(character_filtered.head())

print(f"Length of the dataframe: {len(character_filtered)}")

# Verify the indexes are unique
print(f"Is the indexing unique ? {character_filtered.index.is_unique}")

To optimize, we remove all the characters that are not in the movies dataframe (since we cleaned that):

In [ ]:
# Grab all the movie IDs
movie_ids = movie_df['wiki_ID'].copy(deep=True)

# Merge the two dataframes
dump_characters = character_filtered.reset_index().set_index('wiki_ID')
character_filtered = dump_characters.merge(movie_ids, on='wiki_ID', how='inner')

# Reset the index to before
character_filtered = character_filtered.set_index(['wiki_ID', 'char_name', 'gender'])

print(f"Length of the dataframe: {len(character_filtered)}")
print(f"Is the indexing unique ? {character_filtered.index.is_unique}")

We now create a dataframe with the name of the characetrs and the wikiID of the movie, and their genders, called `name_by_movie_df.csv`

In [ ]:
name_df = character_filtered.reset_index()[['wiki_ID', 'char_name', 'gender']].copy(deep=True)
display(name_df.head())

Let's split the character names and explode it:

In [ ]:
# Split the character names into words and explode the lists
exploded_name_df = name_df.assign(char_words=name_df['char_name'].str.split()).explode('char_words')
word_name_df = exploded_name_df.drop(columns=['char_name'])
display(word_name_df.head())

Now we take care of duplicates for both the characters dataframe (repeated names) and the baby names (keep only one per year), so that we can merge them without any issues.

In [ ]:
# Drop the rows with the same character name
print(f"Size before duplicates removal: {len(word_name_df)}")
word_name_df = word_name_df.drop_duplicates(subset=['wiki_ID', 'gender', 'char_words'])
print(f"Size after duplicates removal: {len(word_name_df)}")

In [ ]:
baby_name_only_df = baby_name_df[['name']].copy(deep=True)

# Does the baby_name_only_df has duplicates?
duplicates = baby_name_only_df.duplicated()
print(f"number of duplicates in baby_name_only_df = {duplicates.sum()}")

# Drop these duplicates
baby_name_only_df = baby_name_only_df.drop_duplicates()
duplicates = baby_name_only_df.duplicated()
print(f"number of duplicates in baby_name_only_df = {duplicates.sum()}")

display(baby_name_only_df.head())

Filter the names to keep only the ones available in the baby name dataset

In [ ]:
# Use pd.merge to filter rows based on 'names' column
word_name_filtered_df = pd.merge(word_name_df.reset_index(), baby_name_only_df, left_on='char_words', right_on='name', how='inner')

word_name_filtered_df.drop(columns=['index', 'name'], inplace=True)
word_name_filtered_df.set_index(['wiki_ID', 'char_words', 'gender'], inplace=True)
print("word_name_filtered_df :")
display(word_name_filtered_df.head())

# Verify the indexes are unique
print(f"Is the indexing unique ? {word_name_filtered_df.index.is_unique}")

In [ ]:
# check for the first movie of the CMU dataset, seems ok
word_name_filtered_df.loc[975900]

The name Lieutenant is still in the filtered baby names dataframe. Let's check if this name is in the baby name dataset

In [ ]:
name_to_search = 'Lieutenant'

# Check if the name is present
is_name_present = name_to_search in baby_name_only_df['name'].values

if is_name_present:
    print(f"{name_to_search} is present in the DataFrame.")
else:
    print(f"{name_to_search} is not present in the DataFrame.")

We save this dataframe in `tmp` so that we can access this data in the next scraping part:

In [ ]:
word_name_filtered_df.reset_index().to_csv(os.path.join(tmp_folder, 'name_by_movie_df.csv'), index=False)

### f. Adding order to the characters in a movie (Scraping part 2)
The website [TMDB claims that](https://www.themoviedb.org/bible/movie/59f3b16d9251414f20000003#59f73ca49251416e7100000e) roles for characters are ordered by importance, namely that major roles are always credited before small parts. 

Therefore make sure you run [this notebook](./scraping/order_characters.ipynb) before continuing. It should have updated the `name_by_movie_ordered_df.csv` file in the `processed` folder, which we will use for our data analysis journey.

## 3. Exporting the remaining dataframes
We save the files that we will use for our data analysis journey:

In [ ]:
display(movie_df.head())
# Export DataFrame to a CSV file in the processed data folder
movie_df.to_csv(os.path.join(processed_folder, 'movie_df.csv'), index=False)

In [ ]:
display(baby_name_with_percentage_df.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
baby_name_with_percentage_df.reset_index().to_csv(os.path.join(processed_folder, 'baby_name_df.csv'), index=False)

We're done with the basic preprocessing! Now we move onto the regression analysis